# Imports

In [2]:
import sys
sys.path.append('/home/beav3r/.local/lib/python3.10/site-packages')

In [7]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

import warnings 
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /home/beav3r/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data

In [8]:
train = pd.read_csv('final_project_train_dataset/train.csv')
train.head()

,review,sentiment
0,I caught this little gem totally by accident b...,positive
1,I can't believe that I let myself into this mo...,negative
2,*spoiler alert!* it just gets to me the nerve ...,negative
3,If there's one thing I've learnt from watching...,negative
4,"I remember when this was in theaters, reviews ...",negative


In [9]:
"""No way, there are no missing values in the dataset. This is a good sign. Let's check the distribution of the target variable."""
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     40000 non-null  object
 1   sentiment  40000 non-null  object
dtypes: object(2)
memory usage: 625.1+ KB


In [10]:
# Setting target variable
target = train.sentiment

In [12]:
"""Wow, there are even no need to visualize it :)"""
# Checking the distribution of the target variable
target.value_counts() / target.shape[0] * 100

sentiment
positive    50.0
negative    50.0
Name: count, dtype: float64